### Imports and Load Environment Variables

In [9]:
import os
import openpyxl
from dotenv import load_dotenv
load_dotenv()

True

### Function to Extract Employee Data from Excel

In [ ]:
def get_all_employee_values(file_path: str, employee_name: str, month: str, column_name: str) -> list:
    """
    Opens an .xlsx workbook and finds all values for a specific employee
    from a given column across ALL sheets that match a given month.

    Args:
        file_path (str): The path to the .xlsx file.
        employee_name (str): The name of the employee to find.
        month (str): The month to search for in sheet names (e.g., "October").
        column_name (str): The header of the column from which to get the value.

    Returns:
        list: A list containing all the values found. The list will be
              empty if no matches are found.
    """
    try:
        workbook = openpyxl.load_workbook(file_path, data_only=True)
        found_values = []

        # Iterate through every sheet in the workbook
        for sheet_name in workbook.sheetnames:
            # If the sheet name doesn't contain the month, skip to the next one
            if month.lower() not in sheet_name.lower():
                continue

            # A sheet for the correct month was found, now process it
            sheet = workbook[sheet_name]

            # Get the header row to find the index of our target column
            header = [cell.value for cell in sheet[1]]
            try:
                col_index = header.index(column_name)
            except ValueError:
                # If the column doesn't exist in this specific sheet, warn the user and skip it
                print(f"Warning: Column '{column_name}' not in sheet '{sheet_name}'. Skipping this sheet.")
                continue

            # Iterate through the data rows to find the employee
            for row in sheet.iter_rows(min_row=2, values_only=True):
                if employee_name in row:
                    value = row[col_index]
                    found_values.append(value)

        return found_values

    except FileNotFoundError:
        print(f"Error: The file at '{file_path}' was not found.")
        return []  # Return an empty list on error
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return [] 

#### Pydantic Models for Structured Output

In [ ]:
from pydantic import BaseModel
from typing import Dict, List

class PersonalReflection(BaseModel):
    what_i_am_most_proud_of: List[str]
    areas_i_am_focused_on_for_growth: List[str]

class WorkAccomplishments(BaseModel):
    goals_of_this_quarter: List[str]
    goals_of_this_month: List[str]
    official_project_accomplishments: List[str]
    personal_project_accomplishments: List[str]
    personal_reflection: PersonalReflection

class EmployeeInfo(BaseModel):
    work_accomplishments: WorkAccomplishments
    learning: List[str]
    utilized_skills: List[str]

#### Create LangChain Agent

In [ ]:
from langchain.agents import create_agent
from pydantic import BaseModel
from langchain_openai import ChatOpenAI

agent = create_agent(
    model = ChatOpenAI(
        model="gemini-2.5-flash-lite",
        base_url=os.getenv('GEMINI_API_BASE_URL'),
        api_key=os.getenv('GEMINI_API_KEY'),
    ),
    tools=[],
    response_format=EmployeeInfo,
    system_prompt="""You are an AI assistant that creates professional brag documents from raw input notes, emphasizing achievements, learning, and utilized skills. Follow these instructions:

**Instructions:**

1. **Input:** Raw notes about accomplishments, learning, and skills.
2. **Output:** A brag document in three sections:
3. **Caution:** Never ever add any irrelavant information which are not in the working history. Genearate only based on the provided employee information.

**Output Structure:**

* **WORK ACCOMPLISHMENTS**
  * Focus on specific results, improvements, or contributions.
  * Use past tense and action-oriented language.
  * Do not add ticket number or Task number (e.g. KR-619 etc.).
  There are five subcategories such as 'Goals of this Quarter', 'Goals of this Month', 'Official Project Accomplishments', 'Personal Project Accomplishments' and 'Personal Reflection'. Inside 'Personal Reflection', there are two subcategories such as 'What I'm Most Proud Of' and 'Areas I'm Focused On For Growth'.



* **LEARNING**

  * Convert raw notes about courses, books, or self-study into bullet points.
  * Highlight practical knowledge, skills gained, or tools explored.

* **UTILIZED SKILLS**

  * Extract technologies, frameworks, or tools used.
  * Present as a comma-separated list or bullets.

**Formatting Rules:**

* Do **not** include a title/header; start with the first section.
* Every line must be a single bullet point.
* Keep language professional, precise, and specific.
* Avoid vague statements; emphasize measurable or observable impact.
* Do not add more than 10 bullet points for any section. Do not add more than 7 skills.
* Use section headers exactly as: `WORK ACCOMPLISHMENTS`, `LEARNING`, `UTILIZED SKILLS`.

    """

)



#### Function to Print Brag Document

In [ ]:
def print_brag_document(response: EmployeeInfo):
    print("WORK ACCOMPLISHMENTS")
    wa = response.work_accomplishments
    print("\nGoals of this Quarter:")
    for item in wa.goals_of_this_quarter:
        print(f"- {item}")
    print("\nGoals of this Month:")
    for item in wa.goals_of_this_month:
        print(f"- {item}")
    print("\nOfficial Project Accomplishments:")
    for item in wa.official_project_accomplishments:
        print(f"- {item}")
    print("\nPersonal Project Accomplishments:")
    for item in wa.personal_project_accomplishments:
        print(f"- {item}")
    print("\nPersonal Reflection:")
    pr = wa.personal_reflection
    print("  What I am Most Proud Of:")
    for item in pr.what_i_am_most_proud_of:
        print(f"  - {item}")
    print("  Areas I am Focused On For Growth:")
    for item in pr.areas_i_am_focused_on_for_growth:
        print(f"  - {item}")

    print("\nLEARNING")
    for item in response.learning:
        print(f"- {item}")

    print("\nUTILIZED SKILLS")
    for item in response.utilized_skills:
        print(f"- {item}")

### Invoke the Agent with Employee Data

In [ ]:
employee_info = get_all_employee_values(file_path='Training 2025 -Daily Scrum Report.xlsx', employee_name='K. M. Abul Farhad-Ibn-Alam', month='September', column_name='What I did  yesterday?')

response = agent.invoke(
    {"messages": [{"role": "user", "content": f"Here is the employee information: {" ".join(employee_info)}"}]}
)

#### Print the Generated Brag Document

In [ ]:
print_brag_document(response=response["structured_response"])